## import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, hamming_loss, confusion_matrix
from sklearn.externals import joblib

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from conf_matrix import pplot_cm
import seaborn as sns

## load data

In [ ]:
data = pd.read_csv("training_data_tweets.csv",sep=',', header=None, names=["sentiment", "text"], index_col=False)

In [ ]:
data

## plot label frequencies

In [ ]:
sns.set(palette = "deep")
sns.countplot(data.sentiment)

## load stopwords

In [ ]:
import io
import unidecode
with io.open('data\\stopwords_german.txt', mode='r', encoding='utf-8') as f:
      content = f.readlines()
content = [x.strip() for x in content]
content = [unidecode.unidecode(x) for x in content]
#content.append('rt')

In [ ]:
content

## split data for training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['sentiment'], test_size=0.1, random_state=1234)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

## set ML pipeline

In [ ]:
pipeline = Pipeline([
    (
        'vect',
        CountVectorizer( 
            analyzer='word',
            strip_accents = 'unicode',
            stop_words=content,
            lowercase=True
        )
    ),
    (
        'tfidf',
        TfidfTransformer(
            use_idf=True,
            smooth_idf=True
        )
    ),
    (
        'rf',
        RandomForestClassifier()
    )
])

pipeline.fit(X_train.values.astype('U'),y_train)

## score model

In [ ]:
def score_model(true, pred):
    print('Accuracy:', accuracy_score(true, pred))
    print('F1:', f1_score(true, pred, average='weighted'))
    print('Precision:', precision_score(true, pred, average='weighted'))
    print('Hamming loss', hamming_loss(true, pred))


score_model(y_test,pipeline.predict(X_test.values.astype('U')))

## plot confusion matrix

In [ ]:
pplot_cm(y_test, pipeline.predict(X_test.values.astype('U')))

## manual tests

In [ ]:
txt_1 = 'Bin rundum begeistert. Insbesondere die Musikwiedergabe ist eine tolle Sache. Die Bedienung lässt keine Wünsche übrig und ist kinderleicht. Beste Grüße, Max'
txt_2 = 'Startet leider nicht mehr. Ich hatte den Echo leider nur ca. 3 Stunden einwandfrei in Betrieb, danach ging er leider nicht mehr. Kann mir vl jemand sagen wie ich ihn wieder zum Laufen bringe oder ob er einfach nur kaputt ist?'
txt_3 = 'Gut, aber im Alltag noch nicht ausgereift. Amazon Echo ist eine praktische Sache allerdings wenn man kein Smart-Home hat ist der Einsatzbereich begrenzt.'
pipeline.predict([txt_3])